# Steps

## Bourso 
- read raw bousoramra
- clean raw bousorama
- make a subsets with only unique : symbol + name combination
- make the mapping of market and the cleaning of symnbol

## Euronext
- read raw euronext
- clean raw euronext
- make a subsets with only unique : Name + ISIN + Symbol + Market combination
- make the mapping of market via the 'Market' column

## Join
- join by unique combination of Symbol + Market ID ?
- join stuff via other techniques
- apply the mapping to the full db bousorama + euronext cleaned

## Import DB
- Import companies table
- Import stocks table
...

In [2]:
import os
import datetime
import dateutil
import tarfile
import pandas as pd
import pandas.io.sql as sqlio
import requests
import re
import glob
import numpy as np

%matplotlib inline
from matplotlib import pylab as plt

## Boursorama

In [3]:
def read_raw_bousorama(year):
    compA = pd.concat({dateutil.parser.parse(f.split('compA ')[1].split('.bz2')[0]):pd.read_pickle(f) for f in glob.glob('/Users/titouanverhille/Developer/pybd-project/reading/bourso/' + year + '/compA*')})
    compB = pd.concat({dateutil.parser.parse(f.split('compB ')[1].split('.bz2')[0]):pd.read_pickle(f) for f in glob.glob('/Users/titouanverhille/Developer/pybd-project/reading/bourso/' + year + '/compB*')})
    merge = pd.concat([compA, compB])
    return merge

def clean_raw_bousorama(df):
    return df

def make_subset_of_companies_bousorama(df):
    # make a copy of whole datframe that contains only the unique combination of symbol + name and only those columns
    df_unique = df[['symbol', 'name']].drop_duplicates(subset=['symbol', 'name']).reset_index(drop=True)
    return df_unique

def add_market_column_boursorama(df):
    # add a column 'Market' to the df_unique and cleaned symbol
    initial_markets_data = (
        (1, "New York", "nyse", "", "NYSE", ""),
        (2, "London Stock Exchange", "lse", "1u*.L", "LSE", ""),
        (3, "Bourse de Milan", "milano", "1g", "", ""),
        (4, "Mercados Espanoles", "mercados", "FF55-", "", ""),
        (5, "Amsterdam", "amsterdam", "1rA", "", "Amsterdam"),
        (6, "Paris", "paris", "1rP", "ENXTPA", "Paris"),
        (7, "Deutsche Borse", "xetra", "1z", "", ""),
        (8, "Bruxelle", "bruxelle", "FF11_", "", "Brussels"),
        (9, "Australie", "asx", "", "ASX", ""),
        (100, "International", "int", "", "", ""),  # should be last one
    )
    df['market'] = df['symbol'].apply(lambda x: next(
        (market[1] for market in initial_markets_data 
         if market[3] and market[3] in x),
        "International"  # Default value if no match found
    ))
    
    # Add cleaned symbol column by removing market prefixes
    df['cleaned_symbol'] = df['symbol'].apply(lambda x: next(
        (x.replace(market[3], '') for market in initial_markets_data 
         if market[3] and market[3] in x),
        x  # Keep original if no prefix found
    ))
    return df


def make_normalized_dataframe_boursorama(df):
    # Columns should be : symbol, name, market and boursorama (for the raw value)
    # rename cleaned_symbol to symbol
    df.rename(columns={'symbol':'boursorama'}, inplace=True)
    df.rename(columns={'cleaned_symbol': 'symbol'}, inplace=True)
    return df


In [ ]:
year = '2021'

raw_boursorama = read_raw_bousorama(year)
raw_boursorama = clean_raw_bousorama(raw_boursorama)
companies_bousorama = make_subset_of_companies_bousorama(raw_boursorama)
companies_bousorama = add_market_column_boursorama(companies_bousorama)
companies_bousorama = make_normalized_dataframe_boursorama(companies_bousorama)
companies_bousorama

In [ ]:
raw_boursorama

## Euronext

In [5]:
def read_raw_euronext(year):
    # raw columns name are
    # Name,ISIN,Symbol,	Market,	Trading, Currency, Open, High, Low, Last, Last Date/Time, Time Zone, Volume, Turnover
    def read_euronext_file(path):
        if path.endswith(".csv"):
            return pd.read_csv(path, delimiter='\t')
        return pd.read_excel(path)

    files = glob.glob('/Users/titouanverhille/Developer/pybd-project/reading/euronext/*' + year + '*')
    df =  pd.concat([read_euronext_file(f) for f in files])
    return df

def clean_raw_euronext(df):
    df = df.iloc[3:]
    df = df[df['Symbol'].notna()]
    return df

def make_subset_of_companies_euronext(df):
    # make a copy of whole datframe that contains only the unique combination of Name	ISIN	Symbol	Market and only those columns
    df_unique = df[['Name', 'ISIN', 'Symbol', 'Market']].drop_duplicates(subset=[
        'Name',
        'ISIN', 
        'Symbol',
        'Market'
    ])
    return df_unique.copy()

def add_market_column_euronext(df):
    # add a column 'Market' to the df_unique
    euronext_market_to_db_market = {
        "Euronext Growth Paris":"Paris",
        "Euronext Paris":"Paris",
        "Euronext Access Paris":"Paris",
        "Euronext Paris, Amsterdam":"Paris",
        "Euronext Paris, Brussels":"Paris",
        "Euronext Amsterdam, Brussels, Paris":"Paris",
        "Euronext Amsterdam, Paris":"Paris",
        "Euronext Brussels, Paris":"Paris",
        "Euronext Growth Paris, Brussels":"Paris",
        "Euronext Paris, Amsterdam, Brussels":"Paris",
        "Euronext Growth Dublin":"Dublin",
        "Euronext Dublin":"Dublin",
    }
    df['market'] = df['Market'].map(euronext_market_to_db_market)
    return df


def make_normalized_dataframe_euronext(df):
    # Columns should be : symbol, name, market and isin
    df.rename(columns={'Symbol': 'symbol'}, inplace=True)
    df.rename(columns={'ISIN': 'isin'}, inplace=True)
    df.rename(columns={'Name': 'name'}, inplace=True)
    # add column euronext with the raw name
    df['euronext'] = df['name']
    return df[['symbol', 'name', 'market', 'isin', 'euronext']]


In [ ]:
year = '2021'

raw_euronext = read_raw_euronext(year)
raw_euronext = clean_raw_euronext(raw_euronext)
companies_euronext = make_subset_of_companies_euronext(raw_euronext)
companies_euronext = add_market_column_euronext(companies_euronext)
companies_euronext = make_normalized_dataframe_euronext(companies_euronext)
companies_euronext


In [ ]:
raw_euronext

## Merge

In [ ]:
# Merge the two DataFrames based on symbol + name + market
merged_companies = pd.merge(companies_euronext, companies_bousorama, 
                            left_on=['symbol', 'name', 'market'], 
                            right_on=['symbol', 'name', 'market'], 
                            suffixes=('_euronext', '_boursorama'),
                            how='outer')

print("original length bourso : ", len(companies_bousorama))
print("original length euronext : ", len(companies_euronext))
print("original length combined : ", len(companies_bousorama) + len(companies_euronext))
print("merged length : ", len(merged_companies))

merged_companies


In [ ]:
merged_companies[merged_companies['name'].str.contains("ACCOR")]


In [ ]:
companies_db_dataframe = pd.DataFrame(
    columns=[
        "id", "name", "mid", "symbol", "isin", "boursorama", "euronext", 
        # not used, ricou explains why in moodle question
        # "pea", "sector1", "sector2", "sector3"
    ],
)

initial_markets_data = (
    (1, "New York", "nyse", "", "NYSE", ""),
    (2, "London Stock Exchange", "lse", "1u*.L", "LSE", ""),
    (3, "Bourse de Milan", "milano", "1g", "", ""),
    (4, "Mercados Espanoles", "mercados", "FF55-", "", ""),
    (5, "Amsterdam", "amsterdam", "1rA", "", "Amsterdam"),
    (6, "Paris", "paris", "1rP", "ENXTPA", "Paris"),
    (7, "Deutsche Borse", "xetra", "1z", "", ""),
    (8, "Bruxelle", "bruxelle", "FF11_", "", "Brussels"),
    (9, "Australie", "asx", "", "ASX", ""),
    (100, "International", "int", "", "", ""),  # should be last one
)
# generate id
companies_db_dataframe['id'] = merged_companies.index
companies_db_dataframe['name'] = merged_companies['name']
# adapt from name (2 columns) to mid (1 column)
companies_db_dataframe['mid'] = merged_companies['market'].apply(lambda x: next(
    (market[1] for market in initial_markets_data 
        if market[3] and market[3] in x),
    "100"  # Default value if no match found
))
companies_db_dataframe['symbol'] = merged_companies['symbol']
companies_db_dataframe['isin'] = merged_companies['isin']
# TODO see to fill column boursorama and euronext
companies_db_dataframe['boursorama'] = merged_companies['boursorama']
companies_db_dataframe['euronext'] = merged_companies['euronext']


companies_db_dataframe


In [ ]:
raw_boursorama

In [ ]:
# add column company_id into raw_boursorama and raw_euronext
# we use maps for optmized performance

bousorama_to_id = dict(zip(companies_db_dataframe['boursorama'], companies_db_dataframe['id']))
raw_boursorama['company_id'] = raw_boursorama['symbol'].map(bousorama_to_id)

euronext_to_id = dict(zip(companies_db_dataframe['euronext'], companies_db_dataframe['id']))
raw_euronext['company_id'] = raw_euronext['Name'].map(euronext_to_id)

raw_euronext


In [ ]:
raw_euronext[raw_euronext['Last Date/Time'] == '-']

In [ ]:
# create daystocks for euronext
daystocks_db_dataframe = pd.DataFrame(
    columns=[
        "date", "cid", "open", "close", "high", "low", "volume", "mean", "std"
    ],
)
daystocks_db_dataframe['raw_date'] = raw_euronext['Last Date/Time']
daystocks_db_dataframe['date'] = pd.to_datetime(raw_euronext['Last Date/Time'], 
                                                format='%d/%m/%y %H:%M',
                                                errors='coerce')
daystocks_db_dataframe['cid'] = raw_euronext['company_id']

# Convert columns to numeric, replacing invalid values with NaN
daystocks_db_dataframe['open'] = pd.to_numeric(raw_euronext['Open'], errors='coerce')
daystocks_db_dataframe['close'] = pd.to_numeric(raw_euronext['Last'], errors='coerce')
daystocks_db_dataframe['high'] = pd.to_numeric(raw_euronext['High'], errors='coerce')
daystocks_db_dataframe['low'] = pd.to_numeric(raw_euronext['Low'], errors='coerce')
daystocks_db_dataframe['volume'] = pd.to_numeric(raw_euronext['Volume'], errors='coerce')
# TODO compute mean and std


# select when date is nan
daystocks_db_dataframe[daystocks_db_dataframe['date'].isna()]

In [ ]:
raw_euronext